In [34]:
import sys
import os
import importlib

# --- 1. 環境判定 ---
IS_COLAB = 'google.colab' in sys.modules

if IS_COLAB:
    print("🌐 Colab環境を検出しました。Jupyter-CadQuery エンジンを構築します...")

    # --- 2. 依存関係の競合を避けたクリーンインストール ---
    !pip uninstall -y websockets google-genai ocp-vscode ocp-cad-viewer jupyter-cadquery
    !pip install -q "websockets>=13.0, <15.0"
    !pip install -q build123d anthropic openai google-genai ipywidgets
    !pip install -q jupyter-cadquery

    # --- 3. リポジトリの展開 (ブランチ 06) ---
    BRANCH = "06"
    REPO_URL = "https://github.com/WOCae/Build123d.git"
    REPO_NAME = "Build123d"

    %cd /content
    if not os.path.exists(REPO_NAME):
        !git clone -b {BRANCH} {REPO_URL}
    else:
        print(f"♻️ 既存の {REPO_NAME} フォルダを使用します。")

    %cd /content/{REPO_NAME}

    # --- 4. viewer.py を Colab/Jupyter-CadQuery 向けに自動調整 (動的パッチ) ---
    viewer_path = "src/gui/utils/viewer.py"
    viewer_code = """
import ipywidgets as w
from IPython.display import display

# Colabで動作するインポートパスを試行
try:
    from jupyter_cadquery import show, PartGroup, Part
except ImportError:
    try:
        from jupyter_cadquery.viewer.client import show
    except ImportError:
        def show(*args, **kwargs):
            print("⚠️ jupyter-cadquery が見つかりません。ランタイムを再起動して再試行してください。")

def _show_viewer(obj, out_widget):
    \"\"\"Build123dオブジェクトを直接Jupyter-CadQueryで描画する\"\"\"
    out_widget.clear_output()
    with out_widget:
        if obj is None:
            print("⚠️ 表示するオブジェクトがありません。")
            return
        try:
            target = obj.part if hasattr(obj, 'part') else obj
            show(target, viewer="standard", height=500)
        except Exception as e:
            print(f"⚠️ 3D表示エラー: {e}")

def _find_latest_stl(code):
    \"\"\"
    古いタブ (manual_tab 等) との互換性を保つためのダミー関数。
    ocp_cad_viewer/jupyter-cadquery 移行後は直接オブジェクトを使用するため、
    ここでは常に None を返します。
    \"\"\"
    return None
"""
    os.makedirs(os.path.dirname(viewer_path), exist_ok=True)
    with open(viewer_path, "w", encoding="utf-8") as f:
        f.write(viewer_code)

    # --- 5. パスの設定 ---
    if os.path.abspath("src") not in sys.path:
        sys.path.insert(0, os.path.abspath("src"))

    print("✅ セットアップ完了！")
    print("⚠️ 正常にインポートできない場合は、上部メニュー [ランタイム] -> [ランタイムを再起動] を押してから、再度このセルを実行してください。")
else:
    print("💻 ローカル環境を検出しました。")

🌐 Colab環境を検出しました。Jupyter-CadQuery エンジンを構築します...
Found existing installation: websockets 15.0.1
Uninstalling websockets-15.0.1:
  Successfully uninstalled websockets-15.0.1
Found existing installation: google-genai 1.64.0
Uninstalling google-genai-1.64.0:
  Successfully uninstalled google-genai-1.64.0
Found existing installation: jupyter_cadquery 4.0.2
Uninstalling jupyter_cadquery-4.0.2:
  Successfully uninstalled jupyter_cadquery-4.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.25.0 requires google-genai<2.0.0,>=1.56.0, which is not installed.
google-adk 1.25.0 requires websockets<16.0.0,>=15.0.1, but you have websockets 14.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.25.0 requ

In [35]:
import build123d_gui

try:
    from google.colab import output
    output.enable_custom_widget_manager()
except ImportError:
    pass

import build123d_gui

try:
    from jupyter_cadquery import set_sidecar
    set_sidecar("Build123d Viewer", False)
except:
    pass

build123d_gui.display_dashboard()

✅ GUI 起動完了  ─  まず「⚙️ API設定」タブでモードとキーを設定してください


In [50]:
import ipywidgets as w
from IPython.display import display
display(w.IntSlider(description="テスト:"))

IntSlider(value=0, description='テスト:')

# Build123d 基本動作確認ノートブック

このノートブックでは、Build123dのインストールと基本的な3Dモデル生成を確認します。

**流れ**
1. インストール
2. 基本形状の生成（Box, Cylinder, Sphere）
3. ブーリアン演算（足し算・引き算）
4. フィレット・面取り
5. 機械部品の例（フランジ付きシャフト）
6. ファイル出力（STEP / STL）
7. LLMプロンプトテンプレート

---
> **LLMを使って3Dモデルを生成したい場合は、セル7のプロンプトをコピーして使ってください。**

## 1. インストール

In [36]:
# build123dのインストール（初回のみ）
import sys
%pip install -q build123d

# Colabの場合、インストール後にランタイム再起動が必要
try:
    from google.colab import runtime
    print('✅ build123d インストール完了')
    print('⚠️  ランタイムを再起動してください（ランタイム → ランタイムを再起動）')
    print('   再起動後はこのセル(セル1)をスキップして、セル2から実行してください。')
except ImportError:
    print('✅ build123d インストール完了（ローカル環境）')


✅ build123d インストール完了
⚠️  ランタイムを再起動してください（ランタイム → ランタイムを再起動）
   再起動後はこのセル(セル1)をスキップして、セル2から実行してください。


In [37]:
# インポートと動作確認
from build123d import *
import os

print("✅ build123d インポート成功")
print(f"   バージョン確認: Box(10,10,10) → {Box(10,10,10)}")

✅ build123d インポート成功
   バージョン確認: Box(10,10,10) → Box at 0x7cafb09b8860, label(), #children(0)


## 2. 基本形状の生成

In [38]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 3. ブーリアン演算（足し算・引き算）

In [39]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [40]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [41]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 4. フィレット・面取り

In [42]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 5. 機械部品の例：フランジ付きシャフト

LLMが生成するコードの典型的な例です。

In [43]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [44]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## 6. ファイル出力（STEP / STL）

- **STEP**: FreeCADやSolidWorksで開ける汎用形式
- **STL**: 3Dプリントや後続のメッシャーへ渡す形式

In [45]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


In [46]:
# ─────────────────────────────────────────────────────────────
# このセルのサンプルコードは 🔬 サンプル タブに移動しました。
# GUIの「🔬 サンプル」タブで ▶ 生成 ボタンを押してください。
# ノートブックの全実行（Run All）では何も生成されません。
# ─────────────────────────────────────────────────────────────


## (参考) 旧プロンプトテンプレート

APIキーをお持ちでない場合、セル7-Bの `LLM_MODE = 'manual'` を使うと  
同等のプロンプトが自動生成されます。このセクションは参考情報です。


In [47]:
def show_prompt_template():
    PROMPT_TEMPLATE = """
あなたはBuild123dのエキスパートです。以下のルールに従ってPythonコードを生成してください。

【ルール】
1. 必ず `from build123d import *` でインポートする
2. 形状は `with BuildPart() as part:` のコンテキストマネージャ内に書く
3. 寸法はすべてmm単位のfloatまたはintで書く
4. パラメータは変数として冒頭にまとめて定義する（変更しやすくするため）
5. ブーリアン演算は mode=Mode.SUBTRACT（穴・引き算）または mode=Mode.ADD（合体）を使う
6. スケッチで穴を作る場合は BuildSketch をネストせず、以下のように mode=Mode.SUBTRACT を使う：
   with BuildSketch() as sk:
       Circle(outer_r)
       Circle(inner_r, mode=Mode.SUBTRACT)
7. フィレットは `fillet(part.edges(), radius=R)` で指定する
8. 最後に以下の2行でファイル出力する：
   export_step(part.part, "output.step")
   export_stl(part.part, "output.stl")
9. コード以外の説明文は不要。コードブロックのみ返す。

【基本形状リファレンス】
- Box(length, width, height)
- Cylinder(radius=R, height=H)
- Sphere(radius=R)
- Cone(bottom_radius=R1, top_radius=R2, height=H)

【スケッチ押し出しリファレンス】
- 断面形状を作ってから押し出す方法（パイプ・複雑断面に使う）:
  with BuildSketch() as sk:
      Circle(outer_r)                        # 外形
      Circle(inner_r, mode=Mode.SUBTRACT)    # 内穴（BuildSketchのネストは使わない）
  extrude(amount=length)

【配置・パターン】
- Locations((x, y, z)): 指定座標に移動
- PolarLocations(radius, count): 円周上に均等配置
- GridLocations(x_spacing, y_spacing, x_count, y_count): グリッド配置

【よくある間違い・注意事項】
- BuildSketch を入れ子（ネスト）にしない
- 穴はすべて mode=Mode.SUBTRACT で指定する
- fillet はすべての形状が確定した後、最後に適用する

【作りたいもの】
"""
    print("📋 以下をLLMにコピー＆ペーストして、最後に作りたいものを追記してください：")
    print("="*60)
    print(PROMPT_TEMPLATE)
    print("="*60)
    print("\n例：【作りたいもの】")
    print("外径60mm、内径40mm、長さ100mmのパイプ。両端にR3のフィレットあり。")

## 8. 手動貼り付け実行セル（manualモード用）

`LLM_MODE = 'manual'` でLLMから返ってきたコードをここに貼り付けて実行してください。  
`LLM_MODE = 'api'` の場合はセル7-Bで自動実行されるため、このセルは不要です。


In [48]:
# ─────────────────────────────────────────────────────────────
# LLMコードの手動実行は 📋 Manual タブに移動しました。
# GUIの「📋 Manual」タブでコードを貼り付けて実行してください。
# ─────────────────────────────────────────────────────────────


---
## 補足メモ

**Colabでの3Dプレビュー**  
Colab上でのインタラクティブな3D表示は現時点では制限があります。  
STLファイルを出力して、別途Three.jsビューワーや外部ツール（MeshLab、FreeCAD等）で確認してください。

**ローカル環境でのプレビュー**  
VS Code + `ocp_vscode` 拡張機能を使うとリアルタイムで3D確認できます。
```
pip install ocp_vscode
```

**後続パーツとの連携**  
- STLファイル → pygmshメッシャーへ渡す  
- STEPファイル → FreeCADやCalculiXのCAEツールへ渡す

**参考リンク**  
- [Build123d 公式ドキュメント](https://build123d.readthedocs.io/)  
- [Build123d GitHub](https://github.com/gumyr/build123d)

In [49]:
# Manualタブでプロンプトテンプレートを表示したい場合は、
# show_prompt_template() を実行してください。
# 例: show_prompt_template()